Q1. The probability of rain on a given calendar day in Vancouver is p[i], where i is the day's index. For
example, p[0] is the probability of rain on January 1 st , and p[10] is the probability of precipitation on January 11 th . Assume
the year has 365 days (i.e., p has 365 elements). What is the chance it rains more than n (e.g., 100) days in Vancouver?
Write a function that accepts p (probabilities of rain on a given calendar day) and n as input arguments and returns the
possibility of raining at least n days.

In [43]:
# assumption_1: the prob of rain of all days are equal and independent
# Applying Binomial distribution : https://en.wikipedia.org/wiki/Binomial_distribution
from scipy.stats import binom
def prob_rain_more_than_n(p, n):
    p1 = p[0]
    N = len(p)
    # calculate 1 - p(X > n)
    return 1 - binom.cdf(n, N, p1)


In [44]:
# Example with N = 5 and n = 2
probabilities = [0.1, 0.1, 0.1, 0.1, 0.1]

n = 2
prob_more_than_n = prob_rain_more_than_n(probabilities, n)
print(f"Probability of raining more than {n} days with equal probabilities: {prob_more_than_n}")

Probability of raining more than 2 days with equal probabilities: 0.008560000000000012


In [9]:
!wget https://raw.githubusercontent.com/tsakim/poibin/master/poibin.py


--2025-02-21 12:04:10--  https://raw.githubusercontent.com/tsakim/poibin/master/poibin.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10547 (10K) [text/plain]
Saving to: ‘poibin.py’

poibin.py           100%[===================>]  10.30K  --.-KB/s    in 0s      

2025-02-21 12:04:10 (84.2 MB/s) - ‘poibin.py’ saved [10547/10547]



In [18]:
# assumption_2: the prob of rain of all days are equal and independent
# Applying Poisson binomial distribution: https://en.wikipedia.org/wiki/Poisson_binomial_distribution
from poibin import PoiBin

def prob_rain_more_than_n_unequal(p, n):
    pb = PoiBin(p)
    return 1 - pb.cdf(n)


In [22]:
# Test with N = 5 and n = 2
probabilities = [0.1, 0.2, 0.05, 0.15, 0.2]
n = 2
prob_more_than_n = prob_rain_more_than_n_unequal(probabilities, n)
print(f"Probability of raining more than {n} days with unequal probabilities: {prob_more_than_n}")

Probability of raining more than 2 days with unequal probabilities: 0.019729999999999137


In [29]:
# solution 2 : implementing Poisson binomial distribution
import itertools
from itertools import permutations,combinations

In [23]:
def find_pos_neg_cases(n,k):
    all_elements = list(range(1,n+1))
    subsets = []
    for r in range(k + 1):
        subsets.extend(combinations(all_elements, r))

    cases_pos = list(map(list, subsets))
    # print(cases_pos)
    cases_neg = []

    for c_pos in cases_pos:
        c_neg = []
        for i in all_elements:
            if i not in c_pos:
                c_neg.append(i)
        cases_neg.append(list(c_neg))
    # print(cases_neg)
    return(cases_pos,cases_neg)

In [24]:
def prob_rain_more_than_n(p:float, n: int) -> float:
    # Assuming that the prob of rain of all days are independent but not equal
    N = len(p)
    k = n
    [pos_cases,neg_cases] = find_pos_neg_cases(N,k)

    total_prob = 0
    for pc,nc in zip(pos_cases,neg_cases):
        prob = 1

        if len(pc) > 0:
          for s in pc:
                     prob *= p[s-1]
        if len(nc) > 0:
          for s in nc:
                     prob *= 1 - p[s-1]
        total_prob += prob
    print(total_prob)

In [25]:
# Test with N = 5 and n = 2
probabilities = [0.1, 0.2, 0.05, 0.15, 0.2]
n = 2
prob_more_than_n = prob_rain_more_than_n_unequal(probabilities, n)
print(f"Probability of raining more than {n} days with unequal probabilities: {prob_more_than_n}")

Probability of raining more than 2 days with unequal probabilities: 0.019729999999999137


Q2: A phoneme is a sound unit (similar to a character for text). We have an extensive pronunciation
dictionary (think millions of words). Below is a snippet: \\
ABACUS​: AE B AH K AH S​ \\
BOOK: B UH K​ \\
THEIR: DH EH R​ \\
THERE: DH EH R​ \\
TOMATO: T AH M AA T OW​ \\
TOMATO: T AH M EY T OW \\
Given a sequence of phonemes as input (e.g. ["DH", "EH", "R", "DH", "EH", "R"]), find all the combinations of the words that
can produce this sequence (e.g. [["THEIR", "THEIR"], ["THEIR", "THERE"], ["THERE", "THEIR"], ["THERE", "THERE"]]). You can
preprocess the dictionary into a different data structure if needed.

In [30]:
# An example for snippets according to the question
snippets = {
    "ABACUS": ["AE", "B", "AH", "K", "AH", "S"],
    "BOOK": ["B", "UH", "K"],
    "THEIR": ["DH", "EH", "R"],
    "THERE": ["DH", "EH", "R"],
    "TOMATO1": ["T", "AH", "M", "AA", "T", "OW"],
    "TOMATO2": ["T", "AH", "M", "EY", "T", "OW"],
}

In [31]:
def find_word_combos_with_pronunciation(phonemes):
    perms = []
    words = []
    for i in range(1, len(phonemes)+1):
        for p in permutations(phonemes, i):
            p = "".join(p)
            perms.append(p)
            for key,val in snippets.items():
                w = "".join(val)
                if w == p:
                    words.append(key)
                    # vals.append(w)
    words = set(words)
    all_words = []

    for i in range(1, len(words)+1):
        for p in permutations(words, i):
            # this statement is added to follow the example output format
            if i == 1:
                all_words.append(list(p)*2)
            else:
                all_words.append(list(p))

    return all_words

In [39]:
# Test
input = ["DH", "EH", "R", "DH", "EH", "R"]
w = find_word_combos_with_pronunciation(input)
print(w)

[['THEIR', 'THEIR'], ['THERE', 'THERE'], ['THEIR', 'THERE'], ['THERE', 'THEIR']]


Q3: Find the n most frequent words in the TensorFlow Shakespeare dataset (https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt).

In [34]:
import requests
import string
from collections import Counter


In [36]:
def find_frequent_words(path, n):
    # Read dataset from the provided url
    response = requests.get(path)
    text = response.text

    # Turn into lower case
    text = text.lower()

    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Split into words
    words = text.split()

    # Count the number of each unique word
    word_counts = Counter(words)

    most_common_words_counts = word_counts.most_common(n)

    most_common_words = [most_common_words_counts[i][0] for i in range(len(most_common_words_counts))]

    return(most_common_words)




In [40]:
# Test with n = 5
url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
find_frequent_words(url,5)

['the', 'and', 'to', 'i', 'of']